## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:26,  8.10it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:40<00:00, 185.68it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,IdeaConnect,"Santa Clara, CA",A Silicon Valley based network connecting disr...,2019-02-18 22:30:48,1315,1702,13294,False,2020-08-18 18:27:42,.@Pontifex @NYGovCuomo @ChrisCuomo @CNN @cnnbr...,[COVID19],Twitter for Android,False
0,Governor Mike DeWine,Ohio,Official Twitter account of Governor Mike DeWi...,2009-01-09 23:40:52,382269,887,269,True,2020-08-18 18:27:42,Watch live at https://t.co/90BcIYqI19 now for ...,[COVID19],Twitter Web App,False
0,1TruthSeeker,Planet Earth,Stand UP. Speak OUT. Share information. 1% vs 99%,2020-04-07 01:31:48,392,363,8103,False,2020-08-18 18:27:42,Talk about paranoid! Imagine how mentally heal...,"[COVID19, Corona]",Twitter for iPhone,False
0,mami olivarri🧬,,"became a mommy to Ava July 9th, 2018 & Greer F...",2015-12-19 23:35:50,151,386,7204,False,2020-08-18 18:27:39,#COVID19 thoughts https://t.co/fq3eTwwjK4,[COVID19],Twitter for iPhone,False
0,Ipsos Public Affairs,,"Ipsos Public Affairs, the global social resear...",2009-05-21 00:38:47,5030,637,340,False,2020-08-18 18:27:38,August marks the 2nd month where the #Ipsos #C...,"[Ipsos, ConsumerConfidence]",Hootsuite Inc.,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (147601, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 147601 all tweets: 155101


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (155101, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)